In [1]:
import pandas as pd

dados = pd.read_csv('/content/carros_simulacao.csv').drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25,
                                                        stratify=y)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [5]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [6]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previcoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previcoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [19]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Acurácia com cross validation em 5 partes = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Acurácia com cross validation em 5 partes = [75.21, 76.35]


In [18]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Acurácia com cross validation em 10 partes = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Acurácia com cross validation em 10 partes = [74.24, 77.32]


## Aleatoriedade no cross validate

In [35]:
def imprime_resultados(results, partes=1):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Acurácia média: %.2f" % (media * 100))
  print("Acurácia no intervalo [%.2f, %.2f] em %d partes" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100, partes))

In [37]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits=10)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results, 10)

Acurácia média: 75.78
Acurácia no intervalo [74.37, 77.19] em 10 partes


In [39]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits=10, shuffle=True)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results, 10)

Acurácia média: 75.78
Acurácia no intervalo [73.58, 77.98] em 10 partes


## Simular situação horrível de azar
* Pode ser um azar de proporção de exemplos, desbalanceado pela classe

In [41]:
dados_azar = dados.sort_values('vendido', ascending=True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [46]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits=10)
results = cross_validate(modelo, x_azar, y_azar, cv=cv)
imprime_resultados(results, 10)

Acurácia média: 57.84
Acurácia no intervalo [34.29, 81.39] em 10 partes


In [47]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = KFold(n_splits=10, shuffle=True)
results = cross_validate(modelo, x_azar, y_azar, cv=cv)
imprime_resultados(results, 10)

Acurácia média: 75.76
Acurácia no intervalo [73.02, 78.50] em 10 partes


In [48]:
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_validate(modelo, x_azar, y_azar, cv=cv)
imprime_resultados(results, 10)

Acurácia média: 75.78
Acurácia no intervalo [74.10, 77.46] em 10 partes


In [58]:
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = StratifiedKFold(n_splits=10)
results = cross_validate(modelo, x_azar, y_azar, cv=cv)
imprime_resultados(results, 10)

Acurácia média: 75.78
Acurácia no intervalo [73.83, 77.73] em 10 partes


## Gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar nosso estimador

In [62]:
SEED = 158020
np.random.seed(SEED)

dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,4


In [63]:
dados.modelo.value_counts()

20    873
19    830
18    788
21    707
17    687
15    669
16    616
22    597
14    577
13    576
12    497
11    397
10    376
23    347
9     320
8     288
7     217
24    199
6     186
5     120
4      66
3      44
2      20
1       3
Name: modelo, dtype: int64

## Testando validação cruzada com GroupKFold

In [64]:
from sklearn.model_selection import GroupKFold

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
cv = GroupKFold(n_splits=10)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo)
imprime_resultados(results, 10)

Acurácia média: 75.76
Acurácia no intervalo [72.91, 78.60] em 10 partes


## Cross validation com StandardScaler

In [67]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 158020
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previcoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previcoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 74.40%


In [69]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

SEED = 158020
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv=cv, groups=dados.modelo)
imprime_resultados(results, 10)

Acurácia média: 76.65
Acurácia no intervalo [73.60, 79.69] em 10 partes


In [71]:
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()
pipeline = Pipeline([('transformador', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x_azar, y_azar, cv=cv, groups=dados.modelo)
imprime_resultados(results, 10)

Acurácia média: 76.66
Acurácia no intervalo [73.65, 79.66] em 10 partes
